# Investigation in ECCO energetics in the CTF

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import scipy.stats
import sys
sys.path.append('/Users/joegradone/SynologyDrive/Drive/Rutgers/Research/code/GitHub/ECCOv4-py')
import ecco_v4_py as ecco
import datetime
import geopandas
from cartopy.mpl.geoaxes import GeoAxes
import cartopy
from sklearn.metrics import mean_squared_error
import cmocean.cm as cmo
import gsw
import geopy.distance
import cartopy.crs as ccrs
import cartopy.feature as cfeature


## Load data

In [2]:
## Load grid
ecco_grid = xr.open_dataset('/Users/joegradone/SynologyDrive/Drive/Rutgers/Research/data/ECCO/ECCO_L4_GEOMETRY_LLC0090GRID_V4R4/GRID_GEOMETRY_ECCO_V4r4_native_llc0090.nc')
ecco_grid = ecco_grid.isel(tile=[10,11])

## Velocity data
ecco_vel = xr.open_mfdataset('/Users/joegradone/SynologyDrive/Drive/Rutgers/Research/data/ECCO/ECCO_L4_OCEAN_VEL_LLC0090GRID_DAILY_V4R4/*.nc',coords='minimal',compat='override')

ecco_vel = ecco_vel.rename({'time':'time_snp'})
## Pull out timestamps
ecco_vel = ecco_vel.isel(time_snp=slice(0,-1))
ecco_vel = ecco_vel.chunk(chunks={'time_snp':1,'tile':1,'k':50,'j':90,'i':90})
ecco_vel

<xarray.Dataset>
Dimensions:    (k_p1: 51, time_snp: 4747, k_l: 50, k: 50, tile: 2, j: 90,
                i_g: 90, k_u: 50, nv: 2, i: 90, j_g: 90, nb: 4)
Coordinates: (12/17)
    Zp1        (k_p1) float32 dask.array<chunksize=(51,), meta=np.ndarray>
    Zu         (k_u) float32 dask.array<chunksize=(50,), meta=np.ndarray>
    Z_bnds     (k, nv) float32 dask.array<chunksize=(50, 2), meta=np.ndarray>
    time_bnds  (time_snp, nv) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    Z          (k) float32 dask.array<chunksize=(50,), meta=np.ndarray>
  * i          (i) int32 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
    ...         ...
  * k_p1       (k_p1) int32 0 1 2 3 4 5 6 7 8 9 ... 42 43 44 45 46 47 48 49 50
  * k_u        (k_u) int32 0 1 2 3 4 5 6 7 8 9 ... 40 41 42 43 44 45 46 47 48 49
  * nb         (nb) float32 0.0 1.0 2.0 3.0
  * nv         (nv) float32 0.0 1.0
  * tile       (tile) int32 10 11
  * time_snp   (time_snp) datetime64[ns] 2005-01-01T12:00:00 ... 2017-12-30T1...
Data variables:
    Zl         (time_snp, k_l) float32 dask.array<chunksize=(1, 50), meta=np.ndarray>
    UVEL       (time_snp, k, tile, j, i_g) float32 dask.array<chunksize=(1, 50, 1, 90, 90), meta=np.ndarray>
    WVEL       (time_snp, k_l, tile, j, i) float32 dask.array<chunksize=(1, 50, 1, 90, 90), meta=np.ndarray>
    VVEL       (time_snp, k, tile, j_g, i) float32 dask.array<chunksize=(1, 50, 1, 90, 90), meta=np.ndarray>
Attributes: (12/63)
    acknowledgement:                 This research was carried out by the Jet...
    author:                          Ian Fenty and Ou Wang
    cdm_data_type:                   Grid
    comment:                         Fields provided on the curvilinear lat-l...
    Conventions:                     CF-1.8, ACDD-1.3
    coordinates_comment:             Note: the global 'coordinates' attribute...
    ...                              ...
    time_coverage_end:               2005-01-02T00:00:00
    time_coverage_resolution:        P1D
    time_coverage_start:             2005-01-01T00:00:00
    title:                           ECCO Ocean Velocity - Daily Mean llc90 G...
    uuid:                            8d337076-416b-11eb-812f-0cc47a3f4807
    history_json:                    [{"$schema":"https:\/\/harmony.earthdata...

## Interpolate onto same grid as other fields

In [3]:
new_grid_delta_lat = 1
new_grid_delta_lon = 1

new_grid_min_lat = -10
new_grid_max_lat = 35

new_grid_min_lon = -100
new_grid_max_lon = -37


lon_shape = len(np.arange(new_grid_min_lon,new_grid_max_lon,new_grid_delta_lon))
lat_shape = len(np.arange(new_grid_min_lat,new_grid_max_lat,new_grid_delta_lat))

depth_len = 23 # 29 for 1000 m, 23 for 500 m

UVEL_interp = np.empty([lat_shape,lon_shape,depth_len,len(ecco_vel.time_snp)])
UVEL_interp[:] = np.nan
VVEL_interp = np.empty([lat_shape,lon_shape,depth_len,len(ecco_vel.time_snp)])
VVEL_interp[:] = np.nan
WVEL_interp = np.empty([lat_shape,lon_shape,depth_len,len(ecco_vel.time_snp)])
WVEL_interp[:] = np.nan


for k in np.arange(0,depth_len):
    for t in np.arange(0,len(ecco_vel.time_snp)):
        new_grid_lon, new_grid_lat, _, _, UVEL_interp[:,:,k,t] = ecco.resample_to_latlon(ecco_grid.XC.values,
                                     ecco_grid.YC.values,
                                     ecco_vel.UVEL.isel(time_snp=t,k=k).compute(),
                                     new_grid_min_lat, new_grid_max_lat, new_grid_delta_lat,
                                     new_grid_min_lon, new_grid_max_lon, new_grid_delta_lon,
                                     fill_value = np.NaN,
                                     mapping_method = 'nearest_neighbor',
                                     radius_of_influence = 120000)

        new_grid_lon, new_grid_lat, _, _, VVEL_interp[:,:,k,t] = ecco.resample_to_latlon(ecco_grid.XC.values,
                                     ecco_grid.YC.values,
                                     ecco_vel.VVEL.isel(time_snp=t,k=k).compute(),
                                     new_grid_min_lat, new_grid_max_lat, new_grid_delta_lat,
                                     new_grid_min_lon, new_grid_max_lon, new_grid_delta_lon,
                                     fill_value = np.NaN,
                                     mapping_method = 'nearest_neighbor',
                                     radius_of_influence = 120000)

        new_grid_lon, new_grid_lat, _, _, WVEL_interp[:,:,k,t] = ecco.resample_to_latlon(ecco_grid.XC.values,
                                     ecco_grid.YC.values,
                                     ecco_vel.WVEL.isel(time_snp=t,k_l=k).compute(),
                                     new_grid_min_lat, new_grid_max_lat, new_grid_delta_lat,
                                     new_grid_min_lon, new_grid_max_lon, new_grid_delta_lon,
                                     fill_value = np.NaN,
                                     mapping_method = 'nearest_neighbor',
                                     radius_of_influence = 120000)

    print('k',k)
    print(datetime.datetime.now())


k 0
2024-06-27 18:32:51.826865
k 1
2024-06-27 20:02:17.434398
k 2
2024-06-27 21:40:04.422029
k 3
2024-06-28 03:58:40.377166
k 4
2024-06-28 07:17:39.421791



KeyboardInterrupt



## Stuff into xarray dataset and safe

In [ ]:
interp_ds = xr.Dataset(
    data_vars=dict(
        uvel=(["lat", "lon", "depth", "time"], UVEL_interp),
        vvel=(["lat", "lon", "depth", "time"], VVEL_interp),
        wvel=(["lat", "lon", "depth", "time"], WVEL_interp),
    ),
    coords=dict(
        lon=new_grid_lon[0,:],
        lat=new_grid_lat[:,0],
        time=ds.time_snp.values,
        depth=ecco_grid.Z[0:depth_len].values
    ),
)

interp_ds


